In [1]:
import numpy
import trax
import trax.fastmath.numpy as np

numpy.random.seed(32)

2023-08-10 10:03:02.259723: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-08-10 10:03:02.722281: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-08-10 10:03:02.722354: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-08-10 10:03:02.722360: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed proper

In [2]:
numpy_array = numpy.random.random((5,10))
print(f"Regular numpy array looks like this \n\n {numpy_array}")
print(f"It is of type: {type(numpy_array)}")

Regular numpy array looks like this 

 [[0.85888927 0.37271115 0.55512878 0.95565655 0.7366696  0.81620514
  0.10108656 0.92848807 0.60910917 0.59655344]
 [0.09178413 0.34518624 0.66275252 0.44171349 0.55148779 0.70371249
  0.58940123 0.04993276 0.56179184 0.76635847]
 [0.91090833 0.09290995 0.90252139 0.46096041 0.45201847 0.99942549
  0.16242374 0.70937058 0.16062408 0.81077677]
 [0.03514717 0.53488673 0.16650012 0.30841038 0.04506241 0.23857613
  0.67483453 0.78238275 0.69520163 0.32895445]
 [0.49403187 0.52412136 0.29854125 0.46310814 0.98478429 0.50113492
  0.39807245 0.72790532 0.86333097 0.02616954]]
It is of type: <class 'numpy.ndarray'>


In [3]:
trax_numpy_array = np.array(numpy_array)
print(f"Trax numpy array looks like this {trax_numpy_array}")
print(f"It is of type: {type(trax_numpy_array)}")

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


Trax numpy array looks like this [[0.8588893  0.37271115 0.55512875 0.9556565  0.7366696  0.81620514
  0.10108656 0.9284881  0.60910916 0.59655344]
 [0.09178413 0.34518623 0.6627525  0.44171348 0.5514878  0.70371246
  0.58940125 0.04993276 0.56179184 0.7663585 ]
 [0.91090834 0.09290995 0.9025214  0.46096042 0.45201847 0.9994255
  0.16242374 0.7093706  0.16062407 0.81077677]
 [0.03514718 0.5348867  0.16650012 0.30841038 0.04506241 0.23857613
  0.67483455 0.7823827  0.69520164 0.32895446]
 [0.49403188 0.52412134 0.29854125 0.46310815 0.9847843  0.50113493
  0.39807245 0.72790533 0.86333096 0.02616954]]
It is of type: <class 'jaxlib.xla_extension.ArrayImpl'>


In [4]:
from trax import layers as tl

# Load from .npy files
predictions = numpy.load('predictions.npy')
targets = numpy.load('targets.npy')

# Cast to jax.interpreters.xla.DeviceArray
predictions = np.array(predictions)
targets = np.array(targets)

# Print shapes
print(f'predictions has shape: {predictions.shape}')
print(f'targets has shape: {targets.shape}')

predictions has shape: (32, 64, 256)
targets has shape: (32, 64)


In [5]:
reshaped_targets = tl.one_hot(targets, predictions.shape[-1])
print(f"reshaped_targets has shape: {reshaped_targets.shape}")

reshaped_targets has shape: (32, 64, 256)


In [6]:
log_p = np.sum(predictions * reshaped_targets, axis = -1)
log_p

Array([[ -5.396545  ,  -1.0311184 ,  -0.66916656, ..., -22.37673   ,
        -23.18771   , -21.843483  ],
       [ -4.5857706 ,  -1.1341286 ,  -8.538033  , ..., -20.15686   ,
        -26.837097  , -23.57502   ],
       [ -5.2223887 ,  -1.2824144 ,  -0.17312431, ..., -21.328228  ,
        -19.854412  , -33.88444   ],
       ...,
       [ -5.396545  , -17.291681  ,  -4.360766  , ..., -20.825802  ,
        -21.065838  , -22.443115  ],
       [ -5.9313164 , -14.247417  ,  -0.2637329 , ..., -26.743248  ,
        -18.38433   , -22.355278  ],
       [ -5.670536  ,  -0.10595131,   0.        , ..., -23.332523  ,
        -28.087376  , -23.878807  ]], dtype=float32)

In [7]:
non_pad = 1.0 - np.equal(targets, 0)
print(f"non_pad has shape: {non_pad.shape}")
print(f"non_pad looks like this \n\n {non_pad}")

non_pad has shape: (32, 64)
non_pad looks like this 

 [[1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 ...
 [1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]]


In [8]:
real_log_p = log_p * non_pad
print(f"real log probabilities still have shape {real_log_p.shape}")

real log probabilities still have shape (32, 64)


In [9]:
print(f'log probabilities before filtering padding: \n\n {log_p}\n')
print(f'log probabilities after filtering padding: \n\n {real_log_p}')

log probabilities before filtering padding: 

 [[ -5.396545    -1.0311184   -0.66916656 ... -22.37673    -23.18771
  -21.843483  ]
 [ -4.5857706   -1.1341286   -8.538033   ... -20.15686    -26.837097
  -23.57502   ]
 [ -5.2223887   -1.2824144   -0.17312431 ... -21.328228   -19.854412
  -33.88444   ]
 ...
 [ -5.396545   -17.291681    -4.360766   ... -20.825802   -21.065838
  -22.443115  ]
 [ -5.9313164  -14.247417    -0.2637329  ... -26.743248   -18.38433
  -22.355278  ]
 [ -5.670536    -0.10595131   0.         ... -23.332523   -28.087376
  -23.878807  ]]

log probabilities after filtering padding: 

 [[ -5.396545    -1.0311184   -0.66916656 ...  -0.          -0.
   -0.        ]
 [ -4.5857706   -1.1341286   -8.538033   ...  -0.          -0.
   -0.        ]
 [ -5.2223887   -1.2824144   -0.17312431 ...  -0.          -0.
   -0.        ]
 ...
 [ -5.396545   -17.291681    -4.360766   ...  -0.          -0.
   -0.        ]
 [ -5.9313164  -14.247417    -0.2637329  ...  -0.          -0.
   -0.  

In [10]:
log_ppx = np.sum(real_log_p, axis=1) / np.sum(non_pad, axis=1)
log_ppx = np.mean(-log_ppx)
print(f"The log perplexity and perplexity of the model are: {log_ppx} and {np.exp(log_ppx)}")

The log perplexity and perplexity of the model are: 2.621185302734375 and 13.75201416015625
